In [1]:
library("dplyr")
library("igraph")
library("scales")
library("ggplot2")
library("circlize")
library("RColorBrewer")
library(tidyverse)
library(viridis)
library(tidyr)
library(patchwork)
library(circlize)
library(SummarizedExperiment)


Attachement du package : ‘dplyr’


Les objets suivants sont masqués depuis ‘package:stats’:

    filter, lag


Les objets suivants sont masqués depuis ‘package:base’:

    intersect, setdiff, setequal, union



Attachement du package : ‘igraph’


Les objets suivants sont masqués depuis ‘package:dplyr’:

    as_data_frame, groups, union


Les objets suivants sont masqués depuis ‘package:stats’:

    decompose, spectrum


L'objet suivant est masqué depuis ‘package:base’:

    union


circlize version 0.4.15
CRAN page: https://cran.r-project.org/package=circlize
Github page: https://github.com/jokergoo/circlize
Documentation: https://jokergoo.github.io/circlize_book/book/

If you use it in published research, please cite:
Gu, Z. circlize implements and enhances circular visualization
  in R. Bioinformatics 2014.

This message can be suppressed by:
  suppressPackageStartupMessages(library(circlize))



Attachement du package : ‘circlize’


L'objet suivant est masqué depuis ‘package:igraph

In [2]:
liana_result <- readRDS("/home/adufour/work/rds_storage/LR/liana/liana_multiomics.rds")

In [3]:
gene_matrix<-readRDS("/home/adufour/work/rds_storage/omics/rna_multiome_stemcells_matrix.rds")

In [4]:
matrix<-as.data.frame(as.matrix(assays(gene_matrix)$GeneExpressionMatrix))

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 4.4 GiB”


In [5]:
rownames(matrix) <- rowData(gene_matrix)$name

In [6]:
liana_result <- subset(liana_result, source %in% c("C2", "C4") & target %in% c("C2", "C4"))

In [7]:
options(repr.plot.width = 18, repr.plot.height = 17, repr.plot.pointsize = 24)

In [8]:
matrix_ligand <- matrix[unique(liana_result$ligand),]
matrix_receptor <- matrix[unique(liana_result$receptor),]

In [9]:
C2_cells <- rownames(as.data.frame(colData(gene_matrix))[as.data.frame(colData(gene_matrix))$Clusters == "C2",])
C4_cells <- rownames(as.data.frame(colData(gene_matrix))[as.data.frame(colData(gene_matrix))$Clusters == "C4",])
C6_cells <- rownames(as.data.frame(colData(gene_matrix))[as.data.frame(colData(gene_matrix))$Clusters == "C6",])

In [10]:
matrix_ligand$C2_means <- rowMeans(matrix_ligand[,C2_cells])
matrix_ligand$C4_means <- rowMeans(matrix_ligand[,C4_cells])
matrix_ligand$C6_means <- rowMeans(matrix_ligand[,C6_cells])

In [11]:
avg_expression_ligands <- matrix_ligand[,c("C2_means", "C4_means", "C6_means")]

In [12]:
sender_ligand_assignment = avg_expression_ligands %>% apply(1, function(ligand_expression){
  ligand_expression > (ligand_expression %>% mean() + ligand_expression %>% sd())
  }) %>% t()
sender_ligand_assignment = sender_ligand_assignment %>% apply(2, function(x){x[x == TRUE]}) %>% purrr::keep(function(x){length(x) > 0})
names(sender_ligand_assignment)

[1] "C2_means" "C4_means" "C6_means"

In [13]:
all_assigned_ligands = sender_ligand_assignment %>% lapply(function(x){names(x)}) %>% unlist()
unique_ligands = all_assigned_ligands %>% table() %>% .[. == 1] %>% names()
general_ligands = liana_result$ligand %>% setdiff(unique_ligands)


C2_specific_ligands = sender_ligand_assignment$C2_means %>% names() %>% setdiff(general_ligands)
C4_specific_ligands = sender_ligand_assignment$C4_means %>% names() %>% setdiff(general_ligands)
C6_specific_ligands = sender_ligand_assignment$C6_means %>% names() %>% setdiff(general_ligands)

ligand_type_indication_df = tibble(
  ligand_type = c(rep("C2-specific", times = C2_specific_ligands %>% length()),
                  rep("C4-specific", times = C4_specific_ligands %>% length()),
                  rep("C6-specific", times = C6_specific_ligands %>% length()),
                  rep("General", times = general_ligands %>% length())),
  ligand = c(C2_specific_ligands, C4_specific_ligands, C6_specific_ligands, general_ligands))

In [14]:
matrix_receptor$C2_means <- rowMeans(matrix_receptor[,C2_cells])
matrix_receptor$C4_means <- rowMeans(matrix_receptor[,C4_cells])
matrix_receptor$C6_means <- rowMeans(matrix_receptor[,C6_cells])

In [15]:
avg_expression_receptor <- matrix_receptor[,c("C2_means", "C4_means", "C6_means")]

In [16]:
sender_receptor_assignment = avg_expression_receptor %>% apply(1, function(receptor_expression){
  receptor_expression > (receptor_expression %>% mean() + receptor_expression %>% sd())
  }) %>% t()
sender_receptor_assignment = sender_receptor_assignment %>% apply(2, function(x){x[x == TRUE]}) %>% purrr::keep(function(x){length(x) > 0})
names(sender_receptor_assignment)

[1] "C2_means" "C4_means" "C6_means"

In [17]:
all_assigned_receptor = sender_receptor_assignment %>% lapply(function(x){names(x)}) %>% unlist()
    unique_receptor = all_assigned_receptor %>% table() %>% .[. == 1] %>% names()
    general_receptor = liana_result$receptor %>% setdiff(unique_receptor)
    
    C2_specific_receptor = sender_receptor_assignment$C2_means %>% names() %>% setdiff(general_receptor)
    C4_specific_receptor = sender_receptor_assignment$C4_means %>% names() %>% setdiff(general_receptor)
    C6_specific_receptor = sender_receptor_assignment$C6_means %>% names() %>% setdiff(general_receptor)
    
    receptor_type_indication_df = tibble(
      receptor_type = c(rep("C2-specific", times = C2_specific_receptor %>% length()),
                      rep("C4-specific", times = C4_specific_receptor %>% length()),
                      rep("C6-specific", times = C6_specific_receptor %>% length()),
                      rep("General", times = general_receptor %>% length())),
      receptor = c(C2_specific_receptor, C4_specific_receptor, C6_specific_receptor, general_receptor))

In [18]:
liana_result <- liana_result[liana_result$source == "C2" & liana_result$ligand %in% C2_specific_ligands | liana_result$source == "C4" & liana_result$ligand %in% C4_specific_ligands | liana_result$source == "C6" & liana_result$ligand %in% C6_specific_ligands | liana_result$ligand %in% general_ligands,]

In [19]:
liana_result <- liana_result[liana_result$target == "C2" & liana_result$receptor %in% C2_specific_receptor | liana_result$target == "C4" & liana_result$receptor %in% C4_specific_receptor | liana_result$target == "C6" & liana_result$receptor %in% C6_specific_receptor | liana_result$receptor %in% general_receptor,]

In [20]:
liana_result$pair <- paste0(liana_result$ligand, "_", liana_result$receptor)

In [21]:
pairs_list <- as.list(liana_result[general_receptor %in% liana_result$receptor | general_ligands %in% liana_result$ligand,]$pair)

Warning message in general_receptor %in% liana_result$receptor | general_ligands %in% :
“la taille d'un objet plus long n'est pas multiple de la taille d'un objet plus court”


In [22]:
colnames(avg_expression_ligands) <- c("C2", "C4", "C6")
colnames(avg_expression_receptor) <- c("C2", "C4", "C6")

In [23]:
mylist <- list()
for (pairs in pairs_list){
    ligand <- unique(liana_result[liana_result$pair == pairs,]$ligand)
    receptor <- unique(liana_result[liana_result$pair == pairs,]$receptor)
    
    if (unique(liana_result[liana_result$pair == pairs,]$ligand) %in% general_ligands) {
        source <- "General"
        mean_ligand <- rowMeans(avg_expression_ligands[ligand,c("C2", "C4")])
    }
    else {
        source <- unique(liana_result[liana_result$pair == pairs,]$source)
        mean_ligand <- avg_expression_ligands[ligand,source]
    }
    if (unique(liana_result[liana_result$pair == pairs,]$receptor) %in% general_receptor) {
        receptor_type <- "General"
        mean_receptor <- rowMeans(avg_expression_receptor[receptor,c("C2", "C4")])
    }
    else {
        receptor_type <- unique(liana_result[liana_result$pair == pairs,]$target)
        mean_receptor <- avg_expression_receptor[receptor,receptor_type]
    }
    line <- c(source, receptor_type, pairs, ligand, receptor, mean_ligand, mean_receptor,
                                               mean(mean_ligand, mean_receptor))
    mylist[[pairs]] <- line
}

In [24]:
liana_result[liana_result$receptor == "ENSSSCG00000038594",]$receptor <- "SDC4"
liana_result[liana_result$receptor == "ENSSSCG00000009347",]$receptor <- "KL"

In [25]:
df <- as.data.frame(do.call("rbind",mylist))

In [26]:
colnames(df) <- c("source", "receptor_type", "pair", "ligand", "receptor", "mean.ligand", "mean.receptor", "mean")

In [27]:
df[df$receptor == "ENSSSCG00000038594",]$receptor <- "SDC4"
df[df$receptor == "ENSSSCG00000009347",]$receptor <- "KL"

In [28]:
df$mean <- as.double(df$mean)

In [29]:
grid_col_ligand =c("General" = "#FDA7DF",
            "C2" = "#00b894",
            "C4" = "#fdcb6e",
            "C6" = "#ce8787")
grid_col_target =c("General" = "#FDA7DF",
            "C2" = "#00b894",
            "C4" = "#fdcb6e",
            "C6" = "#ce8787")

grid_col_tbl_ligand = tibble(source = grid_col_ligand %>% names(), color_ligand_type = grid_col_ligand)
grid_col_tbl_target = tibble(receptor_type = grid_col_target %>% names(), color_target_type = grid_col_target)

#interactions = interactions %>% mutate(ligand = paste(ligand," ")) # extra space: make a difference between a gene as ligand and a gene as target!
df = df %>% inner_join(grid_col_tbl_ligand) %>% inner_join(grid_col_tbl_target)
links_circle = df %>% select(ligand,receptor, mean)
links_circle = links_circle[order(links_circle$mean, decreasing = TRUE),]
links_circle = links_circle[1:30,]

ligand_color = df %>% distinct(ligand,color_ligand_type)
grid_ligand_color = ligand_color$color_ligand_type %>% set_names(ligand_color$ligand)
target_color = df %>% distinct(receptor,color_target_type)
grid_target_color = target_color$color_target_type %>% set_names(target_color$receptor)

grid_col =c(grid_ligand_color,grid_target_color)

# give the option that links in the circos plot will be transparant ~ ligand-target potential score
transparency = df %>% mutate(weight =(mean-min(mean))/(max(mean)-min(mean))) %>% mutate(transparency = 1-mean) %>% .$transparency 

Joining with `by = join_by(source)`
Joining with `by = join_by(receptor_type)`


In [30]:
target_order = c(C2_specific_receptor, C4_specific_receptor,C6_specific_receptor,general_receptor)
ligand_order = c(C2_specific_ligands, C4_specific_ligands,C6_specific_ligands,general_ligands)
order = c(ligand_order,target_order)

In [31]:
order <- gsub("ENSSSCG00000038594","SDC4", order)
order <- gsub("ENSSSCG00000009347","KL", order)

In [32]:
library(svglite)
svglite("~/work/notebook/plots/interactions/omics/stem_cells.svg", width = 30, height = 30)
circos.par(gap.degree = 1)
chordDiagram(links_circle, directional = 1,order=order,link.sort = TRUE, link.decreasing = FALSE, grid.col = grid_col,transparency = 0.25, diffHeight = 0.005, direction.type = c("diffHeight", "arrows"),link.arr.type = "big.arrow",annotationTrack = "grid", 
    preAllocateTracks = list(track.height = 0.075))
# we go back to the first track and customize sector labels
circos.track(track.index = 1, panel.fun = function(x, y) {
    circos.text(CELL_META$xcenter, CELL_META$ylim[1], CELL_META$sector.index,
        facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.55), cex = 3)
}, bg.border = NA)
circos.clear()
dev.off()

png 
  2